In [1]:
import pandas as pd

df = pd.DataFrame(
   {
      "x": [5, 2, 7, 0],
      "y": [4, 7, 5, 1],
      "z": [9, 3, 5, 1]
   }
)
print("Input DataFrame is:\n", df)

col1, col2 = "x", "y"
corr = df[col1].corr(df[col2])
print("Correlation between ", col1, " and ", col2, "is: ", round(corr, 2))

col1, col2 = "x", "x"
corr = df[col1].corr(df[col2])
print("Correlation between ", col1, " and ", col2, "is: ", round(corr, 2))

col1, col2 = "x", "z"
corr = df[col1].corr(df[col2])
print("Correlation between ", col1, " and ", col2, "is: ", round(corr, 2))

col1, col2 = "y", "x"
corr = df[col1].corr(df[col2])
print("Correlation between ", col1, " and ", col2, "is: ", round(corr, 2))

Input DataFrame is:
    x  y  z
0  5  4  9
1  2  7  3
2  7  5  5
3  0  1  1
Correlation between  x  and  y is:  0.41
Correlation between  x  and  x is:  1.0
Correlation between  x  and  z is:  0.72
Correlation between  y  and  x is:  0.41


In [64]:
import pandas as pd

df = pd.read_csv('../../data/GALAUSDT_training_features.csv')
df = df.drop('coin', axis=1)

df['signal'] = df['signal'].replace({'long': 1, 'short': 2, 'dont_trade': 3})

columns = list(df.columns)
columns.pop(columns.index('signal'))

res = [{'feature': column, 'corr': df[column].corr(df['signal'])} for column in columns]

res_df = pd.DataFrame(res)
sorted_res_df = res_df.sort_values(by='corr', ascending=False)
sorted_features = list(sorted_res_df['feature'])


In [65]:
sorted_res_df.head()

,feature,corr
119,DPO_20,0.316619
198,PVT,0.098342
192,PVI_1,0.077847
127,BEARP_13,0.073305
241,SUPERTl_7_3.0,0.070931


In [66]:
sorted_features

['DPO_20',
 'PVT',
 'PVI_1',
 'BEARP_13',
 'SUPERTl_7_3.0',
 'ADOSC_3_10',
 'UO_7_14_28',
 'SQZPRO_ON_WIDE',
 'CMF_20',
 'D_9_3',
 'K_9_3',
 'PSARl_0.02_0.2',
 'ACCBL_20',
 'SQZPRO_ON_NARROW',
 'DCL_20_20',
 'CHOP_14_1_100',
 'HWL',
 'KCLe_20_2',
 'STCstoch_10_12_26_0.5',
 'BBL_5_2.0',
 'J_9_3',
 'THERMOl_20_2_0.5',
 'STOCHd_14_3_3',
 'STOCHk_14_3_3',
 'INERTIA_20_14',
 'QQEs_14_5_4.236',
 'HILOl_13_21',
 'RVI_14',
 'SQZ_NO',
 'SQZPRO_NO',
 'STC_10_12_26_0.5',
 'EBSW_40_10',
 'ABER_XG_5_15',
 'STOCHRSId_14_14_3_3',
 'HA_low',
 'RSX_14',
 'ADX_14',
 'AD',
 'SUPERT_7_3.0',
 'AOBV_LR_2',
 'CTI_12',
 'HWM',
 'HWMA_0.2_0.1_0.1',
 'AROONOSC_14',
 'STOCHRSIk_14_14_3_3',
 'KVOs_34_55_13',
 'MCGD_10',
 'SSF_10_2',
 'HA_open',
 'BBM_5_2.0',
 'MIDPOINT_2',
 'ABER_ZG_5_15',
 'OHLC4',
 'HA_close',
 'HL2',
 'MIDPRICE_2',
 'HLC3',
 'NVI_1',
 'LDECAY_5',
 'JMA_7_0',
 'CDL_SHORTLINE',
 'FWMA_10',
 'EMA_10',
 'HMA_10',
 'PPOs_12_26_9',
 'PWMA_10',
 'TRIMA_10',
 'SWMA_10',
 'SMA_10',
 'RMA_10',
 'HILO_13

In [19]:
a=[list((1,2,'a')),('b',3,4)]
a=(1,2,'a')
b=('b',3,4)
any([11,9]) in list(a)+list(b)

True